In [4]:
from file_management import *
from helpers import *
import pandas as pd
from ridge_reg import *
from build_polynomial import *

In [5]:
ids, predictions, data, features = get_train_higgs_data()
ids.shape, predictions.shape, data.shape, features.shape

((250000, 1), (250000,), (250000, 30), (30,))

In [6]:
#y = pd.Series(predictions)
pred = pd.DataFrame(index = ids.flatten(), data = predictions, columns = ['Predictions'])

In [7]:
df = pd.DataFrame(index=ids.flatten(), data=data, columns=features.flatten())
df_pred = df.join(pred, how = 'inner')

In [8]:
df_NaN = df_pred.replace(-999, np.NaN)
df_NaN.head(10)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Predictions
100000,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,197.760,...,258.733,2.0,67.435,2.150,0.444,46.062,1.240,-2.475,113.497,1
100001,160.937,68.768,103.235,48.146,NaN,NaN,NaN,3.473,2.078,125.157,...,164.546,1.0,46.226,0.725,1.158,NaN,NaN,NaN,46.226,0
100002,NaN,162.172,125.953,35.635,NaN,NaN,NaN,3.148,9.336,197.814,...,260.414,1.0,44.251,2.053,-2.028,NaN,NaN,NaN,44.251,0
100003,143.905,81.417,80.943,0.414,NaN,NaN,NaN,3.310,0.414,75.968,...,86.062,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0
100004,175.864,16.915,134.805,16.405,NaN,NaN,NaN,3.891,16.405,57.983,...,53.131,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0
100005,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,61.619,278.876,...,282.849,3.0,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660,0
100006,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,2.545,305.967,...,294.074,2.0,123.010,0.864,1.450,56.867,0.131,-2.767,179.877,1
100007,154.916,10.418,94.714,29.169,NaN,NaN,NaN,2.897,1.526,138.178,...,187.299,1.0,30.638,-0.715,-1.724,NaN,NaN,NaN,30.638,1
100008,105.594,50.559,100.989,4.288,NaN,NaN,NaN,2.904,4.288,65.333,...,129.804,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0
100009,128.053,88.941,69.272,193.392,NaN,NaN,NaN,1.609,28.859,255.123,...,294.741,1.0,167.735,-2.767,-2.514,NaN,NaN,NaN,167.735,1


In [9]:
jet_zero = df_NaN[df_NaN['PRI_jet_num'] == 0.0]
jet_one = df_NaN[df_NaN['PRI_jet_num'] == 1.0]
jet_two = df_NaN[df_NaN['PRI_jet_num'] == 2.0]
jet_three = df_NaN[df_NaN['PRI_jet_num'] == 3.0]

In [10]:
def drop_nan(x):
    
    drop = []
    for column in x:
        nan_percent = (x[column].isna().sum() / x[column].shape[0]) * 100
        
        if nan_percent > 0:
            print(column, ':' ,nan_percent)

        if nan_percent > 70:
            drop.append(column)

    x_d = x.drop(drop, axis = 1)
    x_n = x_d.fillna(x_d.mean(), inplace=True)

    return x_d        

In [11]:
def drop_low_var(x):
    print(x.var())
    
    drop = []
    for column in x:
        if x[column].var() < 10.0:
            drop.append(column)

    return x.drop(drop, axis = 1)

In [12]:
jet_zero_nonan = drop_nan(jet_zero)
jet_zero_pred = jet_zero_nonan['Predictions']

DER_mass_MMC : 26.145746799715752
DER_deltaeta_jet_jet : 100.0
DER_mass_jet_jet : 100.0
DER_prodeta_jet_jet : 100.0
DER_lep_eta_centrality : 100.0
PRI_jet_leading_pt : 100.0
PRI_jet_leading_eta : 100.0
PRI_jet_leading_phi : 100.0
PRI_jet_subleading_pt : 100.0
PRI_jet_subleading_eta : 100.0
PRI_jet_subleading_phi : 100.0


In [13]:
jet_zero_no_var = drop_low_var(jet_zero_nonan)

DER_mass_MMC                   1977.860756
DER_mass_transverse_met_lep    1024.227287
DER_mass_vis                   1447.320770
DER_pt_h                        278.045852
DER_deltar_tau_lep                0.480658
DER_pt_tot                      278.045851
DER_sum_pt                      555.123393
DER_pt_ratio_lep_tau              0.338254
DER_met_phi_centrality            0.877419
PRI_tau_pt                      231.938904
PRI_tau_eta                       1.521351
PRI_tau_phi                       3.302766
PRI_lep_pt                      212.750782
PRI_lep_eta                       1.718341
PRI_lep_phi                       3.304556
PRI_met                         411.868379
PRI_met_phi                       3.279745
PRI_met_sumet                  2818.187106
PRI_jet_num                       0.000000
PRI_jet_all_pt                    0.000000
Predictions                       0.190046
dtype: float64


In [14]:
jet_one_nonan = drop_nan(jet_one)
jet_one_pred = jet_one_nonan['Predictions']

DER_mass_MMC : 9.751882802022077
DER_deltaeta_jet_jet : 100.0
DER_mass_jet_jet : 100.0
DER_prodeta_jet_jet : 100.0
DER_lep_eta_centrality : 100.0
PRI_jet_subleading_pt : 100.0
PRI_jet_subleading_eta : 100.0
PRI_jet_subleading_phi : 100.0


In [15]:
jet_one_no_var = drop_low_var(jet_one_nonan)

DER_mass_MMC                   3178.371866
DER_mass_transverse_met_lep    1266.021073
DER_mass_vis                   1788.285272
DER_pt_h                       2249.700418
DER_deltar_tau_lep                0.544102
DER_pt_tot                      289.269483
DER_sum_pt                     4248.704462
DER_pt_ratio_lep_tau              0.716436
DER_met_phi_centrality            1.223988
PRI_tau_pt                      441.239441
PRI_tau_eta                       1.484000
PRI_tau_phi                       3.292265
PRI_lep_pt                      455.205799
PRI_lep_eta                       1.593185
PRI_lep_phi                       3.295879
PRI_met                         804.137831
PRI_met_phi                       3.283115
PRI_met_sumet                  6707.796889
PRI_jet_num                       0.000000
PRI_jet_leading_pt             2038.929763
PRI_jet_leading_eta               3.318200
PRI_jet_leading_phi               3.303410
PRI_jet_all_pt                 2038.929765
Predictions

In [16]:
jet_two_nonan = drop_nan(jet_two)
jet_two_pred = jet_two_nonan['Predictions']


DER_mass_MMC : 5.859584350622283


In [17]:
jet_two_no_var = drop_low_var(jet_two_nonan)

DER_mass_MMC                     2925.900504
DER_mass_transverse_met_lep      1244.251095
DER_mass_vis                     1593.377089
DER_pt_h                         4974.379968
DER_deltaeta_jet_jet                3.285597
DER_mass_jet_jet               182202.986695
DER_prodeta_jet_jet                14.600593
DER_deltar_tau_lep                  0.576619
DER_pt_tot                        416.555105
DER_sum_pt                      10273.234535
DER_pt_ratio_lep_tau                1.069975
DER_met_phi_centrality              0.956967
DER_lep_eta_centrality              0.158858
PRI_tau_pt                        783.108472
PRI_tau_eta                         1.374030
PRI_tau_phi                         3.297273
PRI_lep_pt                        722.642985
PRI_lep_eta                         1.424738
PRI_lep_phi                         3.295430
PRI_met                          1618.506132
PRI_met_phi                         3.293661
PRI_met_sumet                   12428.636591
PRI_jet_nu

In [18]:
jet_three_nonan = drop_nan(jet_three)
jet_three_pred = jet_three_nonan['Predictions']


DER_mass_MMC : 6.663959574084101


In [19]:
jet_three_no_var = drop_low_var(jet_three_nonan)

DER_mass_MMC                     4691.739341
DER_mass_transverse_met_lep      1449.708546
DER_mass_vis                     2380.549880
DER_pt_h                         7026.104022
DER_deltaeta_jet_jet                2.160682
DER_mass_jet_jet               100666.499708
DER_prodeta_jet_jet                 8.221607
DER_deltar_tau_lep                  0.684359
DER_pt_tot                       1043.239090
DER_sum_pt                      22541.534001
DER_pt_ratio_lep_tau                1.555411
DER_met_phi_centrality              0.889925
DER_lep_eta_centrality              0.150262
PRI_tau_pt                       1051.759386
PRI_tau_eta                         1.451391
PRI_tau_phi                         3.327540
PRI_lep_pt                       1086.794775
PRI_lep_eta                         1.482754
PRI_lep_phi                         3.305491
PRI_met                          2469.931939
PRI_met_phi                         3.284730
PRI_met_sumet                   25226.661735
PRI_jet_nu

In [22]:
for k in range(jet_zero_no_var.shape[0]):
    x_0_poly = build_poly(jet_zero_no_var[k], 6)
    
    
    
    
    #x_1_poly = build_poly(jet_one_no_var, 6)
    #x_2_poly = build_poly(jet_two_no_var, 6)
    #x_3_poly = build_poly(jet_two_no_var, 6)

KeyError: 0

In [20]:
def ridge_demo(x, pred, lambda_):
    df = x.to_numpy()
    w = ridge_regression(pred, df, lambda_)

    return w

In [41]:
Lambda = 0.10010010010010717
w_zero = ridge_demo(jet_zero_no_var, jet_zero_pred, Lambda)

w_one = ridge_demo(jet_one_no_var, jet_one_pred, Lambda)

w_two = ridge_demo(jet_two_no_var, jet_two_pred, Lambda)
 
w_three = ridge_demo(jet_three_no_var, jet_three_pred, Lambda)

In [85]:
ids_t, predictions_t, data_t, features_t = get_test_higgs_data()
df_t = pd.DataFrame(index=ids_t.flatten(), data=data_t, columns=features_t.flatten())
df_t = df_t.replace(-999, np.NaN)

In [87]:
jet_zero_t = df_t[df_t['PRI_jet_num'] == 0.0]
jet_one_t = df_t[df_t['PRI_jet_num'] == 1.0]
jet_two_t = df_t[df_t['PRI_jet_num'] == 2.0]
jet_three_t = df_t[df_t['PRI_jet_num'] == 3.0]

In [3]:
jet_zero_nonan_t = drop_nan(jet_zero_t)
jet_zero_no_var_t = drop_low_var(jet_zero_nonan_t)

jet_one_nonan_t = drop_nan(jet_one_t)
jet_one_no_var_t = drop_low_var(jet_one_nonan_t)

jet_two_nonan_t = drop_nan(jet_two_t)
jet_two_no_var_t = drop_low_var(jet_two_nonan_t)

jet_three_nonan_t = drop_nan(jet_three_t)
jet_three_no_var_t = drop_low_var(jet_three_nonan_t)

NameError: name 'drop_nan' is not defined

In [2]:
x_0_poly_t = build_poly(jet_zero_no_var_t, 6)
x_1_poly_t = build_poly(jet_one_no_var_t, 6)
x_2_poly_t = build_poly(jet_two_no_var_t, 6)
x_3_poly_t = build_poly(jet_two_no_var_t, 6)

NameError: name 'jet_zero_no_var_t' is not defined

In [89]:
#x_0 = jet_zero_no_var_t.to_numpy()
#x_1 = jet_one_no_var_t.to_numpy()
#x_2 = jet_two_no_var_t.to_numpy()
#x_3 = jet_three_no_var_t.to_numpy()

pred_0 = x_0 @ w_zero
pred_1 = x_1 @ w_one
pred_2 = x_2 @ w_two
pred_3 = x_3 @ w_three

pred_0[pred_0 < 0.5] = -1
pred_0[pred_0 >= 0.5] = 1

pred_1[pred_1 < 0.5] = -1
pred_1[pred_1 >= 0.5] = 1

pred_2[pred_2 < 0.5] = -1
pred_2[pred_2 >= 0.5] = 1

pred_3[pred_3 < 0.5] = -1
pred_3[pred_3 >= 0.5] = 1

In [90]:
preds_0 = pd.Series(pred_0)
preds_1 = pd.Series(pred_1)
preds_2 = pd.Series(pred_2)
preds_3 = pd.Series(pred_3)

preds_0.index = jet_zero_no_var_t.index
preds_1.index = jet_one_no_var_t.index
preds_2.index = jet_two_no_var_t.index
preds_3.index = jet_three_no_var_t.index

new_preds0 = pd.concat([jet_zero_no_var_t, preds_0], axis = 1)
new_preds1 = pd.concat([jet_one_no_var_t, preds_1], axis = 1)
new_preds2 = pd.concat([jet_two_no_var_t, preds_2], axis = 1)
new_preds3 = pd.concat([jet_three_no_var_t, preds_3], axis = 1)

In [103]:
preds = pd.concat([new_preds0[0], new_preds1[0], new_preds2[0], new_preds3[0]], axis = 0).sort_index()
preds


350000   -1.0
350001   -1.0
350002   -1.0
350003    1.0
350004   -1.0
         ... 
918233   -1.0
918234   -1.0
918235    1.0
918236   -1.0
918237   -1.0
Name: 0, Length: 568238, dtype: float64

In [104]:
create_csv_submission(ids_t, preds, 'prediction_3_split')